In [5]:
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os

In [6]:
# load the face detector 
protopath = os.path.sep.join(['./facedetector/', "deploy.prototxt"])
modelpath = os.path.sep.join(['./facedetector/',"res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protopath, modelpath)
print("Face Detector Loaded")

# load the embedding model 
embedder = cv2.dnn.readNetFromTorch('openface_nn4.small2.v1.t7')
print("Embedder Loaded")

Face Detector Loaded
Embedder Loaded


In [7]:
# grab the paths to the input images in our dataset
imagePaths = list(paths.list_images('./dataset'))

knownEmbeddings = []
knownNames = []

# initialize the total number of faces processed
total = 0

for (i, imagePath) in enumerate(imagePaths):
    # extract the person name from the image path
    print("processing image {}/{}".format(i + 1,
        len(imagePaths)))
    name = imagePath.split(os.path.sep)[-2]

    # load the image, resize it to have a width of 600 pixels 
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=600)
    (h, w) = image.shape[:2]

    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()

    # ensure at least one face was found
    if len(detections) > 0:
        # we're making the assumption that each image has only ONE
        # face, so find the bounding box with the largest probability
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]

        # ensure that the detection with the largest probability also
        # means our minimum probability test (thus helping filter out
        # weak detections)
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for
            # the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # extract the face ROI and grab the ROI dimensions
            face = image[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]


            # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vector = embedder.forward()

            # add the name of the person + corresponding face
            # embedding to their respective lists
            knownNames.append(name)
            knownEmbeddings.append(vector.flatten())
            total += 1

# dump the facial embeddings + names to disk
print("Total of {} encodings...".format(total))

data = {"embeddings": knownEmbeddings, "names": knownNames}
print(knownNames)

#store the data
f = open('embeddings.pickle', "wb")
f.write(pickle.dumps(data))
f.close()

processing image 1/238
processing image 2/238
processing image 3/238
processing image 4/238
processing image 5/238
processing image 6/238
processing image 7/238
processing image 8/238
processing image 9/238
processing image 10/238
processing image 11/238
processing image 12/238
processing image 13/238
processing image 14/238
processing image 15/238
processing image 16/238
processing image 17/238
processing image 18/238
processing image 19/238
processing image 20/238
processing image 21/238
processing image 22/238
processing image 23/238
processing image 24/238
processing image 25/238
processing image 26/238
processing image 27/238
processing image 28/238
processing image 29/238
processing image 30/238
processing image 31/238
processing image 32/238
processing image 33/238
processing image 34/238
processing image 35/238
processing image 36/238
processing image 37/238
processing image 38/238
processing image 39/238
processing image 40/238
processing image 41/238
processing image 42/238
p